In [211]:
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
import matplotlib.pyplot as pl
from sklearn.feature_extraction.text import CountVectorizer

%matplotlib inline

In [202]:
train_data = pd.read_csv("linear_train.txt", header=None).values.T

In [203]:
test_data = list(map(lambda x: x[0], np.array(pd.read_csv("linear_test.txt", header=None))))

In [204]:
count_vectorizer = CountVectorizer(lowercase=False, min_df=1, ngram_range=(1, 4), token_pattern='[а-яё]')
train_features = count_vectorizer.fit_transform(train_data[0])

In [205]:
estimator = SGDClassifier(loss='log', penalty='l1')

In [206]:
cross_val_score(estimator, train_features, list(map(int, data[1])), cv=3, scoring='roc_auc').mean()

0.8032340165395051

In [207]:
estimator.fit(train_features, list(map(int, data[1] + 1)))

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l1', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [208]:
test_features = count_vectorizer.transform(test_data)

In [209]:
final_answer = estimator.predict_proba(test_features)

In [210]:
sample_submission = pd.read_csv('linear_ans_example.txt')
sample_submission['Answer'] = final_answer
sample_submission.to_csv("submission.tsv", sep=',', index=False)